In [1]:
import pandas as pd
import bigml.api
from pprint import pprint
api = bigml.api.BigML()

In [3]:
training_df = pd.read_csv('./cs-training.csv')
test_df = pd.read_csv('./cs-test.csv')

dataframes = [training_df, test_df]

In [4]:
for df in dataframes:
    df.columns.values[0]="Id"
    df[['MonthlyIncome','NumberOfDependents']] = df[['MonthlyIncome','NumberOfDependents']].fillna(0)
    df['NumberOfTimes30DaysOrMoreLate'] = df['NumberOfTime30-59DaysPastDueNotWorse']+df['NumberOfTime60-89DaysPastDueNotWorse']+df['NumberOfTimes90DaysLate']
    df['IncomePerPerson'] = df['MonthlyIncome']/(df['NumberOfDependents']+1)
    df['MonthlyDebt'] = df['MonthlyIncome']*df['DebtRatio']
    df['MonthlyBalance'] = df['MonthlyIncome']-df['MonthlyDebt']

In [6]:
training_df.to_csv('./full_train_modif.csv', index=False)
test_df.to_csv('./test_modif.csv', index=False)

In [8]:
full_train_source = api.create_source('./full_train_modif.csv')
api.ok(full_train_source)

True

In [9]:
full_train_dataset = api.create_dataset(full_train_source)
api.ok(full_train_dataset
      )

True

In [10]:
dataset_train80 = api.create_dataset(full_train_dataset,{"name":"Train", "sample_rate":0.8, "seed":"my seed"})
api.ok(dataset_train80)

True

In [11]:
dataset_valid20 = api.create_dataset(full_train_dataset, {"name":"Validation", "sample_rate":0.8, "seed":"my seed", "out_of_bag":True})
api.ok(dataset_valid20)

True

In [12]:
ensemble = api.create_ensemble(dataset_train80, {"objective_field":"SeriousDlqin2yrs"})
api.ok(ensemble)

True

In [14]:
test_source = api.create_source('./test_modif.csv')
api.ok(test_source)

True

In [15]:
test_dataset = api.create_dataset(test_source)
api.ok(test_dataset)

True

In [16]:
batch_prediction = api.create_batch_prediction(ensemble, dataset_valid20, {'all_fields':True, 'probabilities':True})
api.ok(batch_prediction)

True

In [17]:
api.download_batch_prediction(batch_prediction, filename= './my_predictions.csv')

'./results2/my_predictions.csv'

In [18]:
batch_prediction_df = pd.read_csv('./my_predictions.csv', index_col=0)
batch_prediction_df

,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents,NumberOfTimes30DaysOrMoreLate,IncomePerPerson,MonthlyDebt,MonthlyBalance,SeriousDlqin2yrs.1,0 probability,1 probability
Id,,,,,,,,,,,,,,,,,,
5,0,0.907239,49,1,0.024926,63588,7,0,1,0,0,1,63588.0,1584.975094,62003.024906,0,0.83763,0.16237
20,0,0.602794,25,0,0.065868,333,2,0,0,0,0,0,333.0,21.934132,311.065868,0,0.94194,0.05806
27,0,0.052436,58,0,0.097672,8333,22,0,1,0,0,0,8333.0,813.902326,7519.097674,0,0.99112,0.00888
28,0,0.034421,69,0,0.042383,2500,17,0,0,0,1,0,1250.0,105.957617,2394.042382,0,0.99114,0.00886
31,0,0.704074,28,1,0.155201,4200,8,0,0,0,0,1,4200.0,651.844801,3548.155199,0,0.89217,0.10783
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149964,0,0.230353,61,0,883.000000,0,15,0,0,0,2,0,0.0,0.000000,0.000000,0,0.98873,0.01127
149972,0,0.058001,83,0,0.013997,5000,6,0,0,0,0,0,5000.0,69.986005,4930.013995,0,0.99660,0.00340
149976,0,0.022088,58,0,2716.000000,0,8,0,2,0,0,0,0.0,0.000000,0.000000,0,0.99322,0.00678


In [19]:
def error(row):
    if (row['SeriousDlqin2yrs'] == row['SeriousDlqin2yrs.1'] and row['SeriousDlqin2yrs'] == 0):
        row['error'] = 'TN'
    elif (row['SeriousDlqin2yrs'] == row['SeriousDlqin2yrs.1'] and row['SeriousDlqin2yrs'] == 1):
        row['error'] = 'TP'
    elif row['SeriousDlqin2yrs'] > row['SeriousDlqin2yrs.1']:
        row['error'] = 'FN'
    else:
        row['error'] = 'FP'
    return row['error']

batch_prediction_df['error'] = batch_prediction_df.apply(error, axis=1)
batch_prediction_df

,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents,NumberOfTimes30DaysOrMoreLate,IncomePerPerson,MonthlyDebt,MonthlyBalance,SeriousDlqin2yrs.1,0 probability,1 probability,error
Id,,,,,,,,,,,,,,,,,,,
5,0,0.907239,49,1,0.024926,63588,7,0,1,0,0,1,63588.0,1584.975094,62003.024906,0,0.83763,0.16237,TN
20,0,0.602794,25,0,0.065868,333,2,0,0,0,0,0,333.0,21.934132,311.065868,0,0.94194,0.05806,TN
27,0,0.052436,58,0,0.097672,8333,22,0,1,0,0,0,8333.0,813.902326,7519.097674,0,0.99112,0.00888,TN
28,0,0.034421,69,0,0.042383,2500,17,0,0,0,1,0,1250.0,105.957617,2394.042382,0,0.99114,0.00886,TN
31,0,0.704074,28,1,0.155201,4200,8,0,0,0,0,1,4200.0,651.844801,3548.155199,0,0.89217,0.10783,TN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149964,0,0.230353,61,0,883.000000,0,15,0,0,0,2,0,0.0,0.000000,0.000000,0,0.98873,0.01127,TN
149972,0,0.058001,83,0,0.013997,5000,6,0,0,0,0,0,5000.0,69.986005,4930.013995,0,0.99660,0.00340,TN
149976,0,0.022088,58,0,2716.000000,0,8,0,2,0,0,0,0.0,0.000000,0.000000,0,0.99322,0.00678,TN


In [20]:
bp_df_classified = batch_prediction_df.sort_values(by=['error','1 probability'], ascending=[True, True])
bp_df_classified.groupby(['error']).error.count()

error
FN     1683
FP      238
TN    27747
TP      332
Name: error, dtype: int64

In [21]:
options = ['FN', 'FP']
falsy_df = batch_prediction_df[batch_prediction_df['error'].isin(options)]
falsy_df

,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents,NumberOfTimes30DaysOrMoreLate,IncomePerPerson,MonthlyDebt,MonthlyBalance,SeriousDlqin2yrs.1,0 probability,1 probability,error
Id,,,,,,,,,,,,,,,,,,,
89,1,0.734478,45,0,0.591478,4716,13,0,2,0,2,0,1572.000000,2789.408522,1926.591478,0,0.90327,0.09673,FN
227,1,1.953488,38,0,0.153500,3556,5,0,0,0,2,0,1185.333333,545.846501,3010.153499,0,0.84187,0.15814,FN
241,1,0.973339,61,1,0.209962,5500,11,0,0,1,1,2,2750.000000,1154.790037,4345.209963,0,0.69165,0.30835,FN
335,1,0.037198,47,0,0.039758,25000,11,0,1,0,3,0,6250.000000,993.960250,24006.039750,0,0.99634,0.00366,FN
353,0,0.507973,69,2,0.193935,3000,10,2,0,2,0,6,3000.000000,581.806065,2418.193935,1,0.37495,0.62505,FP
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149439,1,0.705074,35,0,0.114565,3377,5,0,0,0,3,0,844.250000,386.885434,2990.114566,0,0.94362,0.05638,FN
149559,1,0.837702,66,0,0.935768,5666,14,0,3,0,0,0,5666.000000,5302.064230,363.935770,0,0.95062,0.04938,FN
149665,1,0.840021,54,0,0.904777,3181,10,0,3,0,0,0,3181.000000,2878.095223,302.904777,0,0.88673,0.11327,FN


In [22]:
#falsy_df['tx_error'] = abs(falsy_df['1 probability'] - 0.5)
falsy_df.loc[:,'tx_error'] = abs(falsy_df['1 probability'] - 0.5)
#falsy_df_classified = falsy_df.sort_values(by=['tx_error'], ascending=False)
#top_100_error = falsy_df_classified[:100]
falsy_df

/opt/conda/lib/python3.7/site-packages/pandas/core/indexing.py:376: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/opt/conda/lib/python3.7/site-packages/pandas/core/indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents,NumberOfTimes30DaysOrMoreLate,IncomePerPerson,MonthlyDebt,MonthlyBalance,SeriousDlqin2yrs.1,0 probability,1 probability,error,tx_error
Id,,,,,,,,,,,,,,,,,,,,
89,1,0.734478,45,0,0.591478,4716,13,0,2,0,2,0,1572.000000,2789.408522,1926.591478,0,0.90327,0.09673,FN,0.40327
227,1,1.953488,38,0,0.153500,3556,5,0,0,0,2,0,1185.333333,545.846501,3010.153499,0,0.84187,0.15814,FN,0.34186
241,1,0.973339,61,1,0.209962,5500,11,0,0,1,1,2,2750.000000,1154.790037,4345.209963,0,0.69165,0.30835,FN,0.19165
335,1,0.037198,47,0,0.039758,25000,11,0,1,0,3,0,6250.000000,993.960250,24006.039750,0,0.99634,0.00366,FN,0.49634
353,0,0.507973,69,2,0.193935,3000,10,2,0,2,0,6,3000.000000,581.806065,2418.193935,1,0.37495,0.62505,FP,0.12505
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149439,1,0.705074,35,0,0.114565,3377,5,0,0,0,3,0,844.250000,386.885434,2990.114566,0,0.94362,0.05638,FN,0.44362
149559,1,0.837702,66,0,0.935768,5666,14,0,3,0,0,0,5666.000000,5302.064230,363.935770,0,0.95062,0.04938,FN,0.45062
149665,1,0.840021,54,0,0.904777,3181,10,0,3,0,0,0,3181.000000,2878.095223,302.904777,0,0.88673,0.11327,FN,0.38673


In [24]:
falsy_df.to_csv('./top_100_error.csv')

In [25]:
evaluation = api.create_evaluation(ensemble, dataset_valid20)
api.ok(evaluation)

True

In [26]:
my_prediction_df = pd.read_csv('./results2/my_predictions.csv', index_col = 0)
my_prediction_df

,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents,NumberOfTimes30DaysOrMoreLate,IncomePerPerson,MonthlyDebt,MonthlyBalance,SeriousDlqin2yrs.1,0 probability,1 probability
Id,,,,,,,,,,,,,,,,,,
5,0,0.907239,49,1,0.024926,63588,7,0,1,0,0,1,63588.0,1584.975094,62003.024906,0,0.83763,0.16237
20,0,0.602794,25,0,0.065868,333,2,0,0,0,0,0,333.0,21.934132,311.065868,0,0.94194,0.05806
27,0,0.052436,58,0,0.097672,8333,22,0,1,0,0,0,8333.0,813.902326,7519.097674,0,0.99112,0.00888
28,0,0.034421,69,0,0.042383,2500,17,0,0,0,1,0,1250.0,105.957617,2394.042382,0,0.99114,0.00886
31,0,0.704074,28,1,0.155201,4200,8,0,0,0,0,1,4200.0,651.844801,3548.155199,0,0.89217,0.10783
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149964,0,0.230353,61,0,883.000000,0,15,0,0,0,2,0,0.0,0.000000,0.000000,0,0.98873,0.01127
149972,0,0.058001,83,0,0.013997,5000,6,0,0,0,0,0,5000.0,69.986005,4930.013995,0,0.99660,0.00340
149976,0,0.022088,58,0,2716.000000,0,8,0,2,0,0,0,0.0,0.000000,0.000000,0,0.99322,0.00678


In [27]:
my_prediction_df.to_csv('./results2/prediction-kaggle.csv')

In [28]:
matrice_confusion = evaluation['object']['result']['model']['confusion_matrix']
matrice_conf_df = pd.DataFrame(matrice_confusion, columns={'P','N'}, index={'P','N'})
pprint(matrice_conf_df)

       P    N
P  27747  238
N   1683  332


In [29]:
auc = evaluation['object']['result']['model']['average_area_under_roc_curve']
pprint(auc)

0.85555


In [30]:
import kaggle
#kaggle.api.competition_submit("./results/prediction-kaggle.csv", "BigML ensemble", "GiveMeSomeCredit")